In [1]:
import gc
import os
import sys
import math
import random
import warnings
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)
import optuna
from tqdm import tqdm
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from pandarallel import pandarallel
# pandarallel.initialize()
warnings.simplefilter("ignore")
import ctypes as ct
from datetime import timedelta

from sklearn.metrics import mean_absolute_error

import statistics as st
import lightgbm as lgbm
from scipy.stats import norm

In [2]:
sys.path.append('../../')
import src.utils as utils

## Param

In [3]:
EXP_NUM = 64
NFOLDS = 5
SEED = 42

In [4]:
utils.set_seed(SEED)

## Dir

In [5]:
DATA_DIR = Path("/home/knikaido/work/MLB-Player-Digital-Engagement-Forecasting/data/")
MAIN_DATA_DIR = DATA_DIR / 'mlb-player-digital-engagement-forecasting'
TRAIN_DIR = MAIN_DATA_DIR / 'train'
OUTPUT_DIR = Path('./output/')

In [6]:
players = pd.read_csv(MAIN_DATA_DIR / 'players.csv')

rosters = pd.read_csv(TRAIN_DIR / 'rosters_train.csv')
targets = pd.read_csv(TRAIN_DIR / 'nextDayPlayerEngagement_train.csv')
scores = pd.read_csv(TRAIN_DIR / 'playerBoxScores_train.csv')
scores = scores.groupby(['playerId', 'date']).sum().reset_index()
seasons = pd.read_csv(MAIN_DATA_DIR / 'seasons.csv')
salaries = pd.read_csv(MAIN_DATA_DIR / 'mlbSalaries.csv')
teams = pd.read_csv(MAIN_DATA_DIR / 'teams.csv')

standings = pd.read_csv(TRAIN_DIR / 'standings_train.csv')
playerTwitterFollowers = pd.read_csv(TRAIN_DIR / 'playerTwitterFollowers_train.csv')

# events = pd.read_csv(TRAIN_DIR / 'events_train.csv')
# events = events.groupby(['gameDate']).sum().reset_index()


In [7]:
train_elements_dict = {"players":players, 
                       "rosters":rosters, 
                       "targets":targets, 
                       "scores":scores, 
                       "seasons":seasons, 
                       "teams":teams, 
                       "standings":standings}

In [8]:
def map_team_name(name):
    names = name.split('-')
    result = ''
    for n in names:
        if n == 'st':
            n = 'st.'
        result += f' {n.capitalize()}'

    return result[1:]

In [9]:
def calc_corr(df):
    # 相関係数行列を作成
    corr_mat = df.corr(method='pearson')

    # 行（列）サイズを取得
    n = corr_mat.shape[0]
    corr_ary = []

    for i in range(n):
        for j in range(i):
            if i == j:
                continue
            corr_ary.append(corr_mat.iloc[i,j])

    return corr_ary

In [10]:
def calc_probs(pid,df,temp):
    to_append=[pid,'','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    targets=['target1','target2','target3','target4']
    z=1
    for target in targets:
        target_prob = temp[target].tolist()
        mean = np.mean(target_prob)
        std = np.std(target_prob)
        median = st.median(target_prob)
        distribution = norm(mean, std)
        min_weight = min(target_prob)
        max_weight = max(target_prob)
        values = list(np.linspace(min_weight, max_weight))
        probabilities = [distribution.pdf(v) for v in values]
        max_value = max(probabilities)
        max_index = probabilities.index(max_value)
        to_append[z]=mean
        to_append[z+1]=median
        to_append[z+2]=std
        to_append[z+3]=min_weight
        to_append[z+4]=max_weight
        to_append[z+5]=temp[target].skew()
        to_append[z+6]=temp[target].kurt()

        z=z+7
    corr_ = calc_corr(temp[['target1', 'target2', 'target3', 'target4']])
    to_append[z:] = corr_  
    df_length = len(df)
    df.loc[df_length] = to_append
    return df

In [11]:
def count_consecutive_items_n_cols(df, col_name_list, output_col):
    cum_sum_list = [
        (df[col_name] != df[col_name].shift(1)).cumsum().tolist() for col_name in col_name_list
    ]
    df[output_col] = df.groupby(
        ["_".join(map(str, x)) for x in zip(*cum_sum_list)]
    ).cumcount() + 1
    return df

In [12]:
def extract_season(date_raw, season_start_end):
    idxes = 0
    for raw in season_start_end.iloc():
        idx_ = ((date_raw >= raw.iloc[0]) & (date_raw <= raw.iloc[1])) * 1
        idxes += idx_
    return idxes

In [13]:
# def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
#     oof_pred = np.zeros(len(y_valid), dtype=np.float32)
#     model = lgbm.LGBMRegressor(**params)
#     model.fit(x_train, y_train, 
#         eval_set=[(x_valid, y_valid)],  
#         early_stopping_rounds=verbose, 
#         verbose=verbose)
#     oof_pred = model.predict(x_valid)
#     oof_pred = np.clip(oof_pred, 0, 100)
#     score = mean_absolute_error(oof_pred, y_valid)
#     print('mae:', score)
#     return oof_pred, model, score

In [14]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, verbose=100):
    def opt(trial):
        params = {
                'random_state': SEED,
                'objective':'mae',
                'n_estimators': 10000,
                'learning_rate': 0.1,
                'max_depth': trial.suggest_int('max_depth', 1, 20),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
                'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
                'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
                'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
                'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
                'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
        }

        model_opt = lgbm.LGBMRegressor(**params)

        model_opt.fit(x_train, y_train, 
            eval_set=[(x_train, y_train), (x_valid, y_valid)],  
            early_stopping_rounds=verbose, 
            verbose=verbose)
        oof_pred = model_opt.predict(x_valid)
        oof_pred = np.clip(oof_pred, 0, 100)
        score = mean_absolute_error(oof_pred, y_valid)
        return -score
    return opt

In [15]:
class Rt4kaidoTrain:
    def __init__(self, usetimelinefeature=False):
        
        self.usetimelinefeature = usetimelinefeature
        self.targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
        self.players_cols = ['playerId', 'primaryPositionName', 'birthCity', 'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight', 'playerForTestSetAndFuturePreds']
        self.rosters_cols = ['playerId', 'teamId', 'status', 'date']
        self.salaries_cols = ['teamId', 'salary', 'year']
        self.standings_cols = ['teamId', 'wildCardRank', 'sportGamesBack', 'date']
        self.transactions_cols = ['playerId', 'transaction_flag', 'date']
        self.stat_cols = ["playerId", "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
                        "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
                        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
                        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt",
                        'tgt1_2_corr', 'tgt1_3_corr', 'tgt2_3_corr', 'tgt1_4_corr', 'tgt2_4_corr', 'tgt3_4_corr']

        self.scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances', 'date']

        self.feature_cols1 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank'] 

        self.feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank'] 

        self.feature_cols3 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank'] 

        self.feature_cols4 = ['week_day', 'annual_day', 'month', 'label_playerId', 'label_primaryPositionName', 'label_teamId', 'label_birthCity',
                        'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank'] 
        
        
        # lightgbm
        self.params1 = {
                    'random_state': SEED,
                    'objective':'mae',
                    'n_estimators': 10000,
                    'learning_rate': 0.1,
                    'max_depth': 5,
                    'min_child_weight': 2,
                    'subsample': 0.8,
                    'colsample_bytree': 0.7,
                    'reg_lambda': 0.2219989029856233,
                    'reg_alpha': 0.003911382788589902,
                    'feature_fraction': 0.8972775633184858,
                    'bagging_fraction': 0.9080490178534212,
                    'bagging_freq': 1
        }


        self.params2 = {
                    'random_state': SEED,
                    'objective':'mae',
                    'n_estimators': 10000,
                    'learning_rate': 0.1,
                    'max_depth': 16,
                    'min_child_weight': 16,
                    'subsample': 0.8,
                    'colsample_bytree': 0.8,
                    'reg_lambda': 0.14788207783997104,
                    'reg_alpha': 0.03142098321958639,
                    'feature_fraction': 0.7277188813412163,
                    'bagging_fraction': 0.7968075536678682,
                    'bagging_freq': 7
        }


        self.params3 = {
                    'random_state': SEED,
                    'objective':'mae',
                    'n_estimators': 10000,
                    'learning_rate': 0.1,
                    'max_depth': 4,
                    'min_child_weight': 8,
                    'subsample': 0.9,
                    'colsample_bytree': 0.8,
                    'reg_lambda': 0.06237286017377033,
                    'reg_alpha': 0.025330255862048567,
                    'feature_fraction': 0.95826235847118,
                    'bagging_fraction': 0.613475530706581,
                    'bagging_freq': 8
        }

        self.params4 = {
                    'random_state': SEED,
                    'objective':'mae',
                    'n_estimators': 10000,
                    'learning_rate': 0.1,
                    'max_depth': 11,
                    'min_child_weight': 4,
                    'subsample': 0.7,
                    'colsample_bytree': 0.6,
                    'reg_lambda': 0.009794699765699343,
                    'reg_alpha': 2.394314077427591,
                    'feature_fraction': 0.5417549198679318,
                    'bagging_fraction': 0.5603631731851066,
                    'bagging_freq': 5
        }

    def make_feature(self, train_elements_dict):

        players = train_elements_dict['players']
        rosters = train_elements_dict['rosters']
        targets = train_elements_dict['targets']
        scores = train_elements_dict['scores']
        seasons = train_elements_dict['seasons']
        teams = train_elements_dict['teams']
        standings = train_elements_dict['standings']

        print('calc target stat ... ', end="")

        ## target stats
        targets_train = targets.merge(rosters[self.rosters_cols], on=['playerId', 'date'], how='left')
        targets_train = targets_train[(targets_train['date'] >= 20210401)]

        playerId_list = targets_train['playerId'].unique()
        player_target_probs = pd.DataFrame(columns = self.stat_cols)  
        for pid in tqdm(playerId_list):
            temp = targets_train[targets_train['playerId'] == pid]
            player_target_stats=calc_probs(pid,player_target_probs,temp)

        teamId_list = targets_train['teamId'].dropna().unique()
        team_target_probs = pd.DataFrame(columns = self.stat_cols)
        for pid in tqdm(teamId_list):
            temp = targets_train[targets_train['teamId'] == pid]
            team_target_stats=calc_probs(pid,team_target_probs,temp)

        team_stat_cols = self.stat_cols
        team_stat_cols = team_stat_cols[:1] + ["team_" + word for word in team_stat_cols[1:]]
        team_target_stats.columns = team_stat_cols

        self.feature_cols1 += self.stat_cols[1:-6]
        self.feature_cols2 += self.stat_cols[1:-6]
        self.feature_cols3 += self.stat_cols[1:-6]
        self.feature_cols4 += self.stat_cols[1:-6]

        self.feature_cols1 += team_stat_cols[1:]
        self.feature_cols2 += team_stat_cols[1:]
        self.feature_cols3 += team_stat_cols[1:]
        self.feature_cols4 += team_stat_cols[1:]

        print('done.')

        print('preprocess ... ', end="")
        ## salaries
        # salaries = salaries.groupby(['year', 'team']).sum()['salary'].reset_index()
        # salaries['team'] = salaries['team'].apply(map_team_name)
        # salaries = salaries.merge(teams, left_on='team', right_on='name', how='inner')
        # salaries = salaries.rename(columns={'id': 'teamId'})

        ## seasons
        seasons = seasons.fillna('0000-00-00')
        for c_ in seasons.columns[1:]:
            seasons[c_] = seasons[c_].str.replace('-', '').astype(int)

        ## players
        players['DOY'] = pd.to_datetime(players['DOB'], format="%Y-%m-%d").dt.year
        players['mlbDebutYear'] = pd.to_datetime(players['mlbDebutDate'], format="%Y-%m-%d").dt.year
        players['DebutAge'] = players['mlbDebutYear'] - players['DOY']

        print('done.')

        print('creat feature ... ', end="")
        # creat feature
        train = targets[self.targets_cols].merge(players[self.players_cols], on=['playerId'], how='left')
        train = train.merge(rosters[self.rosters_cols], on=['playerId', 'date'], how='left')
        train = train.merge(scores[self.scores_cols], on=['playerId', 'date'], how='left')
        train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
        train = train.merge(standings[self.standings_cols], on=['teamId', 'date'], how='left')
        train = train.merge(team_target_stats, how='left', left_on=["teamId"],right_on=["playerId"], suffixes=('', 'team_'))
        date_ = pd.to_datetime(train['date'], format="%Y%m%d")
        train['annual_day'] = (date_ - pd.to_datetime(date_.dt.year, format="%Y")) /  timedelta(days=1)
        train['week_day'] = date_.dt.weekday
        train['month'] = date_.dt.month
        train['year'] = date_.dt.year

        # label encoding
        player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
        position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
        birthCityn2num = {c: i for i, c in enumerate(train['birthCity'].unique())}
        teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
        status2num = {c: i for i, c in enumerate(train['status'].unique())}
        train['label_playerId'] = train['playerId'].map(player2num)
        train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
        train['label_birthCity'] = train['birthCity'].map(birthCityn2num)
        train['label_teamId'] = train['teamId'].map(teamid2num)
        train['label_status'] = train['status'].map(status2num)

        ## season_info
        on_preseason_idxes = extract_season(train['date'], seasons[['preSeasonStartDate', 'preSeasonEndDate']])
        on_season_idxes = extract_season(train['date'], seasons[['regularSeasonStartDate', 'regularSeasonEndDate']]) * 2
        on_postseason_idxes = extract_season(train['date'], seasons[['postSeasonStartDate', 'postSeasonEndDate']]) * 3

        special_days = seasons['lastDate1stHalf'].to_list() + seasons['allStarDate'].to_list() + seasons['firstDate2ndHalf'].to_list()
        special_idxes = 0
        for day in special_days:
            special_idxes += (train['date'] == day) * 4

        on_total_season_idxes = on_preseason_idxes
        on_total_season_idxes[on_season_idxes==2] = 2
        on_total_season_idxes[on_postseason_idxes==3] = 3
        on_total_season_idxes[special_idxes==4] = 4

        train['season_info'] = on_total_season_idxes

        ## only on season
        on_whole_idxes = extract_season(train['date'], seasons[['seasonStartDate', 'seasonEndDate']])
        train = train[on_whole_idxes == 1].reset_index(drop=True)

        # train = train.merge(playerTwitterFollowers, how='left', on=["playerId", 'date'])

        ## game_info
        train['gameday'] = ~train['battingOrder'].isna()*1
        train.sort_values(by=['playerId','date'],inplace=True,ascending=True)

        train=count_consecutive_items_n_cols(train,['playerId','gameday'],'daysSinceLastGame')
        train.loc[train['gameday']==1,'daysSinceLastGame'] = 0

        train_game = train[train['gameday']==1]
        train_last_game = train_game[~train_game.duplicated(subset='playerId', keep='last')][['playerId', 'date']]
        train_last_game.columns = ['playerId', 'lastdate']
        train_player_unique = pd.DataFrame(train['playerId'].unique(), columns=['playerId'])
        train_last_game = pd.merge(train_player_unique, train_last_game, on=['playerId'], how='left' )
        train_last_game = train_last_game.fillna(20171231)

        ## only test_player
        train = train[train['playerForTestSetAndFuturePreds']==True].reset_index(drop=True)

        print('done.')
        
        train_features_dict = {'player_target_stats': player_target_stats,
                                'team_target_stats': team_target_stats,
                                'train_last_game': train_last_game, 
                                'player2num': player2num, 
                                'position2num': position2num, 
                                'birthCityn2num': birthCityn2num,
                                'teamid2num': teamid2num,
                                'status2num': status2num,
                                'feature_cols1': self.feature_cols1,
                                'feature_cols2': self.feature_cols2,
                                'feature_cols3': self.feature_cols3,
                                'feature_cols4': self.feature_cols4
                              }
        
        if self.usetimelinefeature:
            train_features_dict['train_last_game'] = train_last_game
            self.feature_cols1 += ['daysSinceLastGame']
            self.feature_cols2 += ['daysSinceLastGame']
            self.feature_cols3 += ['daysSinceLastGame']
            self.feature_cols4 += ['daysSinceLastGame']

        return train, train_features_dict
    
    def train_and_evaluate(self, train, isgamedayonly=False):
        
        if isgamedayonly:
            train = train[train['gameday'] == 1].reset_index(drop=True)
            
        train_X = train
        train_y = train[['target1', 'target2', 'target3', 'target4']]

        oof = np.zeros(train_y.shape) - 1.0
        y_valids = np.zeros(train_y.shape) - 1.0

        tr_idx = (train['date'].astype(int) < 20210401)
        val_idx = ~tr_idx

        x_train = train_X.loc[tr_idx].reset_index(drop=True)
        y_train = train_y.loc[tr_idx].reset_index(drop=True)
        x_valid = train_X.loc[val_idx].reset_index(drop=True)
        y_valid = train_y.loc[val_idx].reset_index(drop=True)

        study1 = optuna.create_study(direction='maximize')
        study1.optimize(fit_lgbm(x_train[self.feature_cols1], y_train['target1'], 
                                 x_valid[self.feature_cols1], y_valid['target1']), n_trials=100)
        
        study2 = optuna.create_study(direction='maximize')
        study2.optimize(fit_lgbm(x_train[self.feature_cols2], y_train['target2'], 
                                 x_valid[self.feature_cols2], y_valid['target2']), n_trials=100)
        
        study3 = optuna.create_study(direction='maximize')
        study3.optimize(fit_lgbm(x_train[self.feature_cols3], y_train['target3'], 
                                 x_valid[self.feature_cols3], y_valid['target3']), n_trials=100)
        
        study4 = optuna.create_study(direction='maximize')
        study4.optimize(fit_lgbm(x_train[self.feature_cols4], y_train['target4'], 
                                 x_valid[self.feature_cols4], y_valid['target4']), n_trials=100)

#         oof1, model1, score1 = fit_lgbm(
#             x_train[self.feature_cols1], y_train['target1'],
#             x_valid[self.feature_cols1], y_valid['target1'],
# #             self.params1
#         )
#         oof2, model2, score2 = fit_lgbm(
#             x_train[self.feature_cols2], y_train['target2'],
#             x_valid[self.feature_cols2], y_valid['target2'],
# #             self.params2
#         )
#         oof3, model3, score3 = fit_lgbm(
#             x_train[self.feature_cols3], y_train['target3'],
#             x_valid[self.feature_cols3], y_valid['target3'],
# #             self.params3
#         )
#         oof4, model4, score4 = fit_lgbm(
#             x_train[self.feature_cols4], y_train['target4'],
#             x_valid[self.feature_cols4], y_valid['target4'],
# #             self.params4
#         )

#         score = (score1+score2+score3+score4) / 4
#         print(f'score: {score}')

#         oof[val_idx, 0] = oof1
#         oof[val_idx, 1] = oof2
#         oof[val_idx, 2] = oof3
#         oof[val_idx, 3] = oof4
#         y_valids[val_idx, 0] = y_valid['target1'].values
#         y_valids[val_idx, 1] = y_valid['target2'].values
#         y_valids[val_idx, 2] = y_valid['target3'].values
#         y_valids[val_idx, 3] = y_valid['target4'].values

#         mae = mean_absolute_error(y_valids[val_idx, :], oof[val_idx, :])
#         print("mae:", mae)

#         val_idx_num = val_idx[val_idx==True].index.to_list()

#         oof_df = train[self.targets_cols]
#         oof_df.iloc[val_idx_num, 1:5] = oof[val_idx_num, :]

#         models = np.array([model1, model2, model3, model4])

        return study1, study2, study3, study4
            

In [16]:
class Rt4kaidoTest:
    def __init__(self, train_features_dict, models_notgameday, models_gameday, usetimelinefeature=False):
        
        self.usetimelinefeature = usetimelinefeature
        self.train_features_dict = train_features_dict
        self.feature_cols1 = train_features_dict['feature_cols1']
        self.feature_cols2 = train_features_dict['feature_cols2']
        self.feature_cols3 = train_features_dict['feature_cols3']
        self.feature_cols4 = train_features_dict['feature_cols4']
        self.models_notgameday = models_notgameday
        self.models_gameday = models_gameday


        self.test_players_cols = ['playerId', 'primaryPositionName', 'birthCity', 'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight']
        self.test_rosters_cols = ['playerId', 'teamId', 'status']
        self.test_standings_cols = ['teamId', 'wildCardRank', 'sportGamesBack']
        self.test_scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances']

    def test_oneline(self, test_df, sample_prediction_df):
        
        null = np.nan
        true = True
        false = False
        
        sample_prediction_df = sample_prediction_df.reset_index(drop=True)

        # creat dataset
        sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                            .map(lambda x: int(x.split('_')[1]))
        # Dealing with missing values
        if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
            test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
        else:
            test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
            for col in rosters.columns:
                if col == 'playerId': continue
                test_rosters[col] = np.nan

        if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
            test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
        else:
            test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
            for col in scores.columns:
                if col == 'playerId': continue
                test_scores[col] = np.nan

        if test_df['standings'].iloc[0] == test_df['standings'].iloc[0]:
            test_standings = pd.DataFrame(eval(test_df['standings'].iloc[0]))
        else:
            test_standings = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
            for col in standings.columns:
                if col == 'playerId': continue
                test_standings[col] = np.nan

        test_scores = test_scores.groupby('playerId').sum().reset_index()
        test = sample_prediction_df[['playerId']].copy()
        test = test.merge(players[self.test_players_cols], on='playerId', how='left')
        test = test.merge(test_rosters[self.test_rosters_cols], on='playerId', how='left')
        test = test.merge(test_scores[self.test_scores_cols], on='playerId', how='left')
        test = test.merge(self.train_features_dict['player_target_stats'], how='left', left_on=["playerId"],right_on=["playerId"])
        test = test.merge(test_standings[self.test_standings_cols], on='teamId', how='left')
        test = test.merge(self.train_features_dict['team_target_stats'], how='left', left_on=["teamId"],right_on=["playerId"], suffixes=('', 'team_'))
        test['wildCardRank'] = test['wildCardRank'].astype(float)


        test['label_playerId'] = test['playerId'].map(self.train_features_dict['player2num'])
        test['label_primaryPositionName'] = test['primaryPositionName'].map(self.train_features_dict['position2num'])
        test['label_teamId'] = test['teamId'].map(self.train_features_dict['teamid2num'])
        test['label_status'] = test['status'].map(self.train_features_dict['status2num'])
        test['label_birthCity'] = test['birthCity'].map(self.train_features_dict['birthCityn2num'])

        date_ = pd.to_datetime(test_df.index[0], format="%Y%m%d")
        test['annual_day'] = (date_ - pd.to_datetime(date_.year, format="%Y")) /  timedelta(days=1)
        test['week_day'] = date_.weekday()
        test['month'] = date_.month
        test['season_info'] = 2
        test['gameday'] = ~test['battingOrder'].isna()*1
        
        if self.usetimelinefeature:
            test['date'] = test_df.index[0]
            test = pd.merge(test, self.train_features_dict['train_last_game'], on=['playerId'], how='left')
            test['daysSinceLastGame'] = (pd.to_datetime(test['date'], format="%Y%m%d") - pd.to_datetime(test['lastdate'], format="%Y%m%d")).dt.days
            
            self.train_features_dict['train_last_game'] = pd.merge(self.train_features_dict['train_last_game'], test[test['gameday']==1][['playerId','date']], on=['playerId'], how='left')
            self.train_features_dict['train_last_game']['lastdate'].update(self.train_features_dict['train_last_game']['date'])
            self.train_features_dict['train_last_game'] = self.train_features_dict['train_last_game'][['playerId', 'lastdate']]

        test_gameday = test[test['gameday']==1]

        if len(test_gameday) != 0:
            gameday_index = list(test_gameday.index)

            test_X = test.iloc[gameday_index]

            pred1 = self.models_gameday[0].predict(test_X[self.feature_cols1])
            pred2 = self.models_gameday[1].predict(test_X[self.feature_cols2])
            pred3 = self.models_gameday[2].predict(test_X[self.feature_cols3])
            pred4 = self.models_gameday[3].predict(test_X[self.feature_cols4])

            # merge submission
            sample_prediction_df['target1'].iloc[gameday_index] = np.clip(pred1, 0, 100)
            sample_prediction_df['target2'].iloc[gameday_index] = np.clip(pred2, 0, 100)
            sample_prediction_df['target3'].iloc[gameday_index] = np.clip(pred3, 0, 100)
            sample_prediction_df['target4'].iloc[gameday_index] = np.clip(pred4, 0, 100)

        # ------------------------------------------------------------

        test_notgameday = test[test['gameday']==0]
        if len(test_notgameday) != 0:
            notgameday_index = list(test_notgameday.index)

            test_X = test.iloc[notgameday_index]

            pred1 = self.models_notgameday[0].predict(test_X[self.feature_cols1])
            pred2 = self.models_notgameday[1].predict(test_X[self.feature_cols2])
            pred3 = self.models_notgameday[2].predict(test_X[self.feature_cols3])
            pred4 = self.models_notgameday[3].predict(test_X[self.feature_cols4])

            # merge submission
            sample_prediction_df['target1'].iloc[notgameday_index] = np.clip(pred1, 0, 100)
            sample_prediction_df['target2'].iloc[notgameday_index] = np.clip(pred2, 0, 100)
            sample_prediction_df['target3'].iloc[notgameday_index] = np.clip(pred3, 0, 100)
            sample_prediction_df['target4'].iloc[notgameday_index] = np.clip(pred4, 0, 100)

        sample_prediction_df = sample_prediction_df.fillna(0.)

        del sample_prediction_df['playerId']
        
        return sample_prediction_df

In [17]:
rt4kaido_train = Rt4kaidoTrain(usetimelinefeature=True)
train, train_features_dict = rt4kaido_train.make_feature(train_elements_dict)

calc target stat ... 

100%|██████████| 30/30 [00:00<00:00, 42.45it/s]


done.
preprocess ... done.
creat feature ... done.


In [18]:
study1, study2, study3, study4 = rt4kaido_train.train_and_evaluate(train, isgamedayonly=False)

[I 2021-07-11 22:49:45,808] A new study created in memory with name: no-name-82439038-ecb6-4645-a40c-86ca23b4ba5b


[LightGBM] [Warning] feature_fraction is set=0.5185086545177852, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.5185086545177852
[LightGBM] [Warning] bagging_fraction is set=0.8038804886103567, subsample=0.6 will be ignored. Current value: bagging_fraction=0.8038804886103567
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.886493	valid_1's l1: 1.21532
[200]	training's l1: 0.886401	valid_1's l1: 1.21512
[300]	training's l1: 0.886397	valid_1's l1: 1.21512


[I 2021-07-11 22:49:57,900] Trial 0 finished with value: -1.2151098931859292 and parameters: {'max_depth': 15, 'min_child_weight': 18, 'subsample': 0.6, 'colsample_bytree': 0.7, 'reg_lambda': 0.0010331376287454664, 'reg_alpha': 40.07273757567476, 'feature_fraction': 0.5185086545177852, 'bagging_fraction': 0.8038804886103567, 'bagging_freq': 4}. Best is trial 0 with value: -1.2151098931859292.


Early stopping, best iteration is:
[216]	training's l1: 0.8864	valid_1's l1: 1.21512
[LightGBM] [Warning] feature_fraction is set=0.8236993457253146, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.8236993457253146
[LightGBM] [Warning] bagging_fraction is set=0.7552391422620561, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7552391422620561
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.878396	valid_1's l1: 1.20154
[200]	training's l1: 0.875587	valid_1's l1: 1.19689
[300]	training's l1: 0.875455	valid_1's l1: 1.19678


[I 2021-07-11 22:50:08,423] Trial 1 finished with value: -1.196752757432271 and parameters: {'max_depth': 6, 'min_child_weight': 10, 'subsample': 0.9, 'colsample_bytree': 0.6, 'reg_lambda': 0.007381428981383331, 'reg_alpha': 0.08205495656828862, 'feature_fraction': 0.8236993457253146, 'bagging_fraction': 0.7552391422620561, 'bagging_freq': 9}. Best is trial 1 with value: -1.196752757432271.


...

Early stopping, best iteration is:
[593]	training's l1: 0.903929	valid_1's l1: 1.57214
[LightGBM] [Warning] feature_fraction is set=0.6068002604616095, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.6068002604616095
[LightGBM] [Warning] bagging_fraction is set=0.7726971750655394, subsample=0.8 will be ignored. Current value: bagging_fraction=0.7726971750655394
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.00147	valid_1's l1: 1.59599
[200]	training's l1: 0.99042	valid_1's l1: 1.58857
[300]	training's l1: 0.972708	valid_1's l1: 1.58427
[400]	training's l1: 0.943423	valid_1's l1: 1.57568
[500]	training's l1: 0.917748	valid_1's l1: 1.56529
[600]	training's l1: 0.903918	valid_1's l1: 1.56846
Early stopping, best iteration is:
[505]	training's l1: 0.916075	valid_1's l1: 1.56448


[I 2021-07-12 00:53:22,502] Trial 98 finished with value: -1.5642766590545805 and parameters: {'max_depth': 20, 'min_child_weight': 12, 'subsample': 0.8, 'colsample_bytree': 0.6, 'reg_lambda': 0.04864835209130865, 'reg_alpha': 0.10189164913608396, 'feature_fraction': 0.6068002604616095, 'bagging_fraction': 0.7726971750655394, 'bagging_freq': 7}. Best is trial 68 with value: -1.5613323278454823.


[LightGBM] [Warning] feature_fraction is set=0.6032921761672233, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.6032921761672233
[LightGBM] [Warning] bagging_fraction is set=0.7706659018521553, subsample=0.8 will be ignored. Current value: bagging_fraction=0.7706659018521553
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.00077	valid_1's l1: 1.5929
[200]	training's l1: 0.985569	valid_1's l1: 1.58516
[300]	training's l1: 0.961743	valid_1's l1: 1.5797
[400]	training's l1: 0.94161	valid_1's l1: 1.58447


[I 2021-07-12 00:53:40,538] Trial 99 finished with value: -1.577465401686825 and parameters: {'max_depth': 13, 'min_child_weight': 12, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_lambda': 0.04762077185070463, 'reg_alpha': 0.3763926216656374, 'feature_fraction': 0.6032921761672233, 'bagging_fraction': 0.7706659018521553, 'bagging_freq': 7}. Best is trial 68 with value: -1.5613323278454823.


Early stopping, best iteration is:
[349]	training's l1: 0.952669	valid_1's l1: 1.57763


In [26]:
print(f'param = {study1.best_trial.params}')
print(f'best_score = {-study1.best_value}')

param = {'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.6, 'colsample_bytree': 0.5, 'reg_lambda': 0.713345675706146, 'reg_alpha': 0.035486851631028014, 'feature_fraction': 0.9590493698301378, 'bagging_fraction': 0.9017962785461695, 'bagging_freq': 3}
best_score = 1.1458399417968985


In [27]:
print(f'param = {study2.best_trial.params}')
print(f'best_score = {-study2.best_value}')

param = {'max_depth': 18, 'min_child_weight': 8, 'subsample': 0.5, 'colsample_bytree': 0.8, 'reg_lambda': 54.45539908757693, 'reg_alpha': 1.4416542297175678, 'feature_fraction': 0.5270058209606729, 'bagging_fraction': 0.7607796738825863, 'bagging_freq': 8}
best_score = 2.2215053088120085


In [28]:
print(f'param = {study3.best_trial.params}')
print(f'best_score = {-study3.best_value}')

param = {'max_depth': 4, 'min_child_weight': 9, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_lambda': 6.674872391777479, 'reg_alpha': 0.06024628993289382, 'feature_fraction': 0.8615761806234242, 'bagging_fraction': 0.6008278034175709, 'bagging_freq': 6}
best_score = 0.9266460044641004


In [29]:
print(f'param = {study4.best_trial.params}')
print(f'best_score = {-study4.best_value}')

param = {'max_depth': 18, 'min_child_weight': 6, 'subsample': 0.5, 'colsample_bytree': 0.5, 'reg_lambda': 0.0031425316581200785, 'reg_alpha': 0.0048645247952439135, 'feature_fraction': 0.5321025519455179, 'bagging_fraction': 0.8302452807177191, 'bagging_freq': 5}
best_score = 1.5613323278454823
